# Instructions

## Formatting

### Ledger

### Bank

#Reconciliation

## 1. Function Setup

In [ ]:
def Reconciliation():
  import pandas as pd
  import numpy as np
  from tqdm import tqdm
  import calendar




  ledger_df = pd.read_excel('/content/Ledger.xlsx')
  ledger_df['Debit'] = ledger_df['Debit'].replace({' ': 0})
  ledger_df['Credit'] = ledger_df['Credit'].replace({' ': 0})
  ledger_df['Debit'] = ledger_df['Debit'].astype('float')
  ledger_df['Credit'] = ledger_df['Credit'].astype('float')
  ledger_df['Date'] = ledger_df['Date'].replace({pd.NaT: 0})
  ledger_df = ledger_df[ledger_df['Date'] != 0]
  ledger_df['Date'] = pd.to_datetime(ledger_df['Date'], dayfirst = False)
  ledger_df.reset_index(drop=True, inplace = True)


  #gets the month name from the numerical month value
  month_string = calendar.month_name[(pd.to_datetime(ledger_df['Date'].iloc[0])).month]


  bank_df = pd.read_excel('/content/Bank.xlsx')
  bank_df['Debit'] = bank_df['Debit'].fillna(0)
  bank_df['Credit'] = bank_df['Credit'].fillna(0)
  bank_df['Debit'] = bank_df['Debit'].replace({' ': 0})
  bank_df['Credit'] = bank_df['Credit'].replace({' ': 0})
  bank_df['Debit'] = bank_df['Debit'].astype('float')
  bank_df['Credit'] = bank_df['Credit'].astype('float')
  bank_df.rename(columns = {'Posting Date': 'Date'}, inplace = True)
  bank_df['Date'] = bank_df['Date'].replace({pd.NaT: 0})
  bank_df = bank_df[bank_df['Date'] != 0]
  bank_df['Date'] = pd.to_datetime(bank_df['Date'], dayfirst = True)

  bank_month_string = calendar.month_name[(pd.to_datetime(bank_df['Date'].iloc[0])).month]


  if month_string != bank_month_string:
    print("Error: Month mismatch between the Ledger and the Bank Statement. Check the uploaded files and try again.")
  else:
    #first pass

    print("Starting First Pass.\n")

    bank_amounts = []
    ledger_amounts = []

    for row in range(0, len(bank_df)):
      bank_amounts += [(abs(bank_df['Debit'].iloc[row]) + abs(bank_df['Credit'].iloc[row]))]
    bank_df['Amount'] = bank_amounts

    for row in range(0, len(ledger_df)):
      ledger_amounts += [abs(ledger_df['Debit'].iloc[row]) + abs(ledger_df['Credit'].iloc[row])]
    ledger_df['Amount'] = ledger_amounts

    ledger_initial_length = len(ledger_df)
    bank_initial_length = len(bank_df)
    print("Ledger Entries to Match", ledger_initial_length,"\nBank Entries to Match", bank_initial_length, "\n")


    ledger_rows_to_drop = []
    bank_rows_to_drop = []

    print("Matching in progress:")
    for ledger_row in tqdm(range(len(ledger_df))):

        for bank_row in range(len(bank_df)):
          """
          print("ledger_row:", ledger_row, "bank_row:", bank_row)
          print("ledger_df['Date'].iloc[ledger_row]:", ledger_df['Date'].iloc[ledger_row])
          print("bank_df['Date'].iloc[bank_row]:", bank_df['Date'].iloc[bank_row])
          print("ledger_df['Amount'].iloc[ledger_row]:", ledger_df['Amount'].iloc[ledger_row])
          print("bank_df['Amount'].iloc[bank_row]:", bank_df['Amount'].iloc[bank_row])
          """
          if bank_row in bank_rows_to_drop:
            continue
          else:

            if ledger_df['Date'].iloc[int(ledger_row)] == bank_df['Date'].iloc[int(bank_row)]:
                if ledger_df['Amount'].iloc[ledger_row] == bank_df['Amount'].iloc[bank_row]:
                    ledger_rows_to_drop += [ledger_row]
                    bank_rows_to_drop += [bank_row]
                    #exit the inner loop after a match is found to avoid unnecessary iterations
                    break
                else:
                    #print("Amounts don't match.")
                    continue
            else:
                #print("Dates don't match.")
                continue

    for row in ledger_rows_to_drop:
      ledger_df = ledger_df.drop(index=row)
    ledger_df.reset_index(drop=True, inplace = True)
    for row in bank_rows_to_drop:
      bank_df = bank_df.drop(index=row)
    bank_df.reset_index(drop=True, inplace = True)

    ledger_current_length = len(ledger_df)
    bank_current_length = len(bank_df)

    ledger_total_matched = ledger_initial_length - ledger_current_length
    bank_total_matched = bank_initial_length - bank_current_length

    print("\nTotal Ledger Entries Matched",ledger_total_matched,"\nTotal Bank Entries Matched", bank_total_matched)

    print("\nTotal Remaining Unmatched Ledger Entries",ledger_current_length,"\nTotal Remaining Unmatched Bank Entries", bank_current_length)

    #second pass
    ledger_initial_length = len(ledger_df)
    bank_initial_length = len(bank_df)

    print("\n----------------------------------------------------\n")
    print("\nStarting Second Pass.")
    print("\nLedger Entries to Match", ledger_initial_length,"\nBank Entries to Match", bank_initial_length, "\n")
    rows_to_drop_ledger = []
    rows_to_drop_bank = []

    print("Matching Unique Date Instances in progress:")
    #iterate over unique dates in ledger_df
    for date in tqdm(ledger_df['Date'].unique()):
        #filter rows with the same date
        same_date_rows = ledger_df[ledger_df['Date'] == date]

        #iterate over unique names in same_date_rows
        for name in same_date_rows['Name'].unique():
            # Filter rows with the same name
            same_name_rows = same_date_rows[same_date_rows['Name'] == name]

            #iterate over rows to find matching combinations
            for i in range(len(same_name_rows)):
                for j in range(i + 1, len(same_name_rows) + 1):
                    combo = same_name_rows.iloc[i:j]
                    combo_sum = combo['Amount'].sum()

                    #find matching rows in bank_df
                    matching_rows = bank_df[bank_df['Amount'] == combo_sum]

                    #check if there are matching rows
                    if len(matching_rows) > 0:
                        # Store the rows to drop
                        rows_to_drop_ledger.extend(combo.index)
                        rows_to_drop_bank.extend(matching_rows.index)


    ledger_df = ledger_df.drop(index=rows_to_drop_ledger).reset_index(drop=True)
    bank_df = bank_df.drop(index=rows_to_drop_bank).reset_index(drop=True)

    ledger_current_length = len(ledger_df)
    bank_current_length = len(bank_df)

    ledger_total_matched = ledger_initial_length - ledger_current_length
    bank_total_matched = bank_initial_length - bank_current_length

    print("\nTotal Ledger Entries Matched",ledger_total_matched,"\nTotal Bank Entries Matched", bank_total_matched)

    print("\nTotal Remaining Unmatched Ledger Entries",ledger_current_length,"\nTotal Remaining Unmatched Bank Entries", bank_current_length)


    unmatched_ledger_entries = ledger_df.copy()
    unmatched_bank_entries = bank_df.copy()
    unmatched_ledger_entries['Date'] = unmatched_ledger_entries['Date'].dt.strftime('%d-%m-%Y')
    unmatched_bank_entries['Date'] = unmatched_bank_entries['Date'].dt.strftime('%d-%m-%Y')




    print("\n----------------------------------------------------\n")
    print("\nCreating Excel Files:")


    names = ["unmatched_ledger_entries","unmatched_bank_entries"]
    dataframes = {
      "unmatched_ledger_entries": unmatched_ledger_entries,
      "unmatched_bank_entries": unmatched_bank_entries
                  }
    """

    for name, element in tqdm(range(0, len(names))):
    #for name, df in tqdm(dataframes.items()):
      filename = f"{dataframes[element][0]}.xlsx"
      dataframe = dataframes[element][0]
      dataframe.to_excel(filename, index=False)
      print(f"Created Excel file: {filename}")"""

    dataframes = {
        "unmatched_ledger_entries": unmatched_ledger_entries,
        "unmatched_bank_entries": unmatched_bank_entries
    }

    names = list(dataframes.keys())

    for i in tqdm(range(len(names))):
      name = names[i]
      df = dataframes[name]
      filename = f"{name}_{month_string}.xlsx"
      df.to_excel(filename, index=False)

    print("\n\nExcel files created successfully.")



## 2. Run

In [ ]:
Reconciliation()

Starting First Pass.

Ledger Entries to Match 630 
Bank Entries to Match 644 

Matching in progress:


100%|██████████| 630/630 [00:09<00:00, 65.50it/s] 



Total Ledger Entries Matched 519 
Total Bank Entries Matched 519

Total Remaining Unmatched Ledger Entries 111 
Total Remaining Unmatched Bank Entries 125

----------------------------------------------------


Starting Second Pass.

Ledger Entries to Match 111 
Bank Entries to Match 125 

Matching Unique Date Instances in progress:


100%|██████████| 16/16 [00:00<00:00, 95.71it/s]



Total Ledger Entries Matched 93 
Total Bank Entries Matched 44

Total Remaining Unmatched Ledger Entries 18 
Total Remaining Unmatched Bank Entries 81

----------------------------------------------------


Creating Excel Files:


100%|██████████| 2/2 [00:00<00:00, 34.77it/s]



Excel files created successfully.
